# Graphs and tables for the weekly fundamentals review

![alt text](https://www.theice.com/publicdocs/images/ICE_Clearing_L2_HeaderImage_Web_1200x250_Sky_20160114.jpg)

_HTTP and API calls to industry data sources to make graphs and analysis for the IFED weekly review_

In [1]:
import Quandl
import numpy as np
import pandas as pd
import datetime as dt  
from dateutil.relativedelta import *
import calendar
import requests
from pprint import pprint
import json

In [35]:
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn
from bokeh.io import output_notebook, show, vform
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [3]:
pd.options.display.float_format = '{:.3f}'.format
float_formatter = lambda x: "%.3f" % x
#np.set_printoptions(formatter={'float_kind':float_formatter})

### Notes

* [relative delta](http://dateutil.readthedocs.org/en/latest/index.html) in the dateutil package is awesome
* [COT](https://www.quandl.com/data/CFTC?keyword=) is updated on tuesdays
* [EIA](http://www.eia.gov/opendata/) API documentation and links
* [Requests](http://docs.python-requests.org/en/master/) HTTP for Python
* [EIA data with python](https://quantcorner.wordpress.com/2014/11/18/downloading-eias-data-with-python/)
* [tweepy](http://docs.tweepy.org/en/v3.5.0/index.html) documentation
    * [twitter example 1](http://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/word_cloud/Mining_Twitter_Example.ipynb)
    * [twitter example 2](https://gist.github.com/arulrajnet/0b71842b573d81c7bc23)
    * also search github git for examples and saved a notbook in my python folder
* Yet another awesome thing in Python [Group By](http://pandas.pydata.org/pandas-docs/stable/groupby.html#applying-multiple-functions-at-once)
* [Periodic table graph](http://bokeh.pydata.org/en/latest/docs/gallery/periodic.html) in Bokeh

### Improvements

* store the commodityMonths dict somewhere I can just load it for regular use
* should pull more data with NG prices query and make a chart or graph form it. The NG query is set up to do convergence, but that requires ICE data and I cant automate with that so make something to replace.
* be able to do multiple weeks for 'this week in H' table

### Set Up variables 

In [4]:
#from datetime get today's date
today = dt.date.today()

#today string for display
today_str = today.strftime("%m/%d/%Y")

#sixty days ago for COT data
sixty = today + relativedelta(months=-2)

#six years ago for benchmarking EIA data
sixyears = today + relativedelta(years=-6)

In [5]:
#last fri for NG price call from quandl
lastFriday = today + relativedelta(weekday=FR(-1))

#last mon for NG price call from quandl
lastMonday = today + relativedelta(weekday=MO(-2))

#picking months from last wed instead of mon prevents picking weeks with zero prompt month prices
#this could be more thought out, or it'd be nice to have a multi year bidweek calendar
lastWednesday = today + relativedelta(weekday=WE(-1))

In [6]:
#dict of commodity months
commodityMonths = {'January':'F','February':'G','March':'H','April':'J','May':'K','June':'M',
'July':'N','August':'Q','September':'U','October':'V','November':'X','  December':'Z'}

In [7]:
#prompt month is the next month, str version used for labeling columns
prompt = lastWednesday + relativedelta(months=+1)
promptStr = calendar.month_name[prompt.month]

#prompt +1 second delivery month
second = lastWednesday + relativedelta(months=+2)
secondStr = calendar.month_name[second.month]

#prompt +3 third delivery month
third = lastWednesday + relativedelta(months=+3)
thirdStr = calendar.month_name[third.month]

In [8]:
#used to authomate symbols in quandl query
promptYear = prompt.year
secondYear = second.year
thirdYear = third.year

In [9]:
#used to automate symbols in quandl query
promptC = commodityMonths[promptStr]
secondC = commodityMonths[secondStr]
thirdC = commodityMonths[thirdStr]

In [10]:
# vars for getting price data
#begin and end date of data
bw = lastMonday.strftime("%m/%d/%Y")
ew = lastFriday.strftime("%m/%d/%Y")

#symbols change month letter and year when necessary
first = "CME/NG"+promptC+str(promptYear)+".6"
second = "CME/NG"+secondC+str(secondYear)+".6"
third = "CME/NG"+thirdC+str(thirdYear)+".6"

#rename change month name when necessary
firstn = "NG " + promptStr
secondn = "NG" + secondStr
thirdn = "NG" + thirdStr

#vars for getting OI data
sixty = sixty.strftime("%m/%d/%Y")

### Get data from quandl

In [11]:
#the .6 after the contract name gives specific column
NG = Quandl.get([first,second,third], authtoken="UDaCz3fqxxJUAL3-WWmD", trim_start=sixty, returns="pandas")

In [12]:
# prompt month NG
NGp = Quandl.get("CHRIS/CME_NG1", authtoken="UDaCz3fqxxJUAL3-WWmD",trim_start=sixyears,returns="pandas")

In [13]:
#leave out trim end defaults to most recent
COT = Quandl.get("CFTC/H_FO_ALL", authtoken="UDaCz3fqxxJUAL3-WWmD",trim_start=sixty, returns="pandas")

### H this week

In [14]:
#copy because quandl has a call limit and sometimes and I want this dataset to be my base.
NG1 = NGp.copy()
NG1.index.name = None

In [15]:
NG1.tail(1)

,Open,High,Low,Last,Change,Settle,Volume,Open Interest
2016-03-23,1.867,1.899,1.777,1.789,0.069,1.794,117041.000,53365.000


In [16]:
#using these to filter later

#get week number of last week in the data
wk = NG1.tail(1).index.week
wk = wk[0]

#get year of last year in the data
yr = NG1.tail(1).index.year
yr = yr[0]

In [17]:
#column for week of year, and year
NG1['week'] = NG1.index.week
NG1['year'] = NG1.index.year

In [18]:
# want to calc some weekly stats and bench mark stats
gwy = NG1.groupby(['week','year'])
gw = NG1.groupby('week')

#lambda functions to call on either gw group or gwy
gMax = lambda x: (x.max())
gMin = lambda x: (x.min())
gMean = lambda x: (x.mean())

#weekly max and min, 
# from pandas docs "The transform method returns an object that is indexed the same (same size) as the one being grouped."
NG1['wyMax'] = gwy['High'].transform(gMax)
NG1['wyMin'] = gwy['Low'].transform(gMin)

#weekly range,weekly begin, end
NG1['wRange'] = abs(NG1['wyMax']-NG1['wyMin'])
NG1['wSettle'] = gwy['Settle'].nth(-1)
NG1['wSettle'] = NG1['wSettle'].fillna(method='bfill')
NG1['wStart'] = gwy['Settle'].nth(0)
NG1['wStart'] = NG1['wStart'].fillna(method='ffill')

NG1['percChng'] = (NG1['Settle']-NG1['Open'])/NG1['wStart']

#five year average settle for that week, range, max price, min
NG1['wxAvgSettle'] = gw['wSettle'].transform(gMean)
NG1['wxAvgRange'] = gw['wRange'].transform(gMean)
NG1['wxMaxSettle'] = gw['wSettle'].transform(gMax)
NG1['wxMinSettle'] = gw['wSettle'].transform(gMin)


In [19]:
lastWeek = gwy.get_group((wk,yr)).tail(1)
lastWeek_short = lastWeek[['Open','High','Low','Last','Settle','wRange','percChng']]
lastWeek_short

,Open,High,Low,Last,Settle,wRange,percChng
2016-03-23,1.867,1.899,1.777,1.789,1.794,0.131,-0.040


In [20]:
lastWeek_comp = lastWeek[['week','wxAvgSettle','wxAvgRange','wxMaxSettle','wxMinSettle']]
lastWeek_comp.set_index('week',inplace=True)
lastWeek_comp

,wxAvgSettle,wxAvgRange,wxMaxSettle,wxMinSettle
week,,,,
12,3.370,0.227,4.403,1.794


## Table display H weekly

In [91]:
#im trying to get the month name out of the data

NG1['promptMonth'] = NG1.index.month
calendar.month_name[NG1['promptMonth'][0]]

NG1['monthName'] = NG1['promptMonth'].apply(lambda x: calendar.month_name[x])

#df['year'] = df['date'].apply(lambda t: t[0:4])
#calendar.month_name

In [151]:
NGgraph = NG1.copy()

In [152]:
NGgraph = NGgraph[['week','year','wRange','wSettle','percChng','wxAvgSettle','wxAvgRange','monthName']]

In [153]:
NGgraph = NGgraph.resample('1W',how='last', fill_method='pad')

In [154]:
NGgraph['wSettle'] = NGgraph['wSettle'].apply(lambda x: float_formatter(x))
NGgraph['wRange'] = NGgraph['wRange'].apply(lambda x: float_formatter(x))
NGgraph['percChng'] = NGgraph['percChng'].apply(lambda x: float_formatter(x))
NGgraph['wxAvgSettle'] = NGgraph['wxAvgSettle'].apply(lambda x: float_formatter(x))
NGgraph['wxAvgRange'] = NGgraph['wxAvgRange'].apply(lambda x: float_formatter(x))

In [163]:
NGgraph.dtypes

week            int32
year            int32
wRange         object
wSettle        object
percChng       object
wxAvgSettle    object
wxAvgRange     object
monthName      object
dtype: object

In [155]:
# array of years, last 6 right now, bottom will be oldest
rowLabel = NGgraph.year.unique()
rowLabel = [str(x) for x in rowLabel]

In [156]:
# array of weeks may want to cut this to fewer weeks but starting with all 52
# colLabel = NGgraph.week.unique()
# colLabel.sort()
# colLabel = [str(x) for x in colLabel]
colLabel = [str(x) for x in range(1, 19)]

In [157]:
#color each year
colormap = {
    "2010" : "#a6cee3",
    "2011" : "#1f78b4",
    "2012" : "#fdbf6f", 
    "2013" : "#b2df8a",
    "2014" : "#33a02c", 
    "2015" : "#bbbb88",
    "2016" : "#baa2a6",
}

In [158]:
# arrange data in dict for graphing 
source = ColumnDataSource(
    data=dict(
       
        # week will be x axis, year will be y axis of graph
        week=[str(x) for x in NGgraph['week']],
        year=[str(x) for x in NGgraph['year']],
        
        # positioning for box display stats
        settleX=[str(x)+":0.1" for x in NGgraph['week']],
        monthY=[str(x)+":0.8" for x in NGgraph["year"]],
        pctY=[str(x)+":0.15" for x in NGgraph["year"]],
        avgY=[str(x)+":0.3" for x in NGgraph["year"]],
        
        # display stats and hover over stats
        settle=NGgraph['wSettle'],
        month=NGgraph['monthName'],
        pctChng=NGgraph['percChng'],
        wxAvgSettle=NGgraph['wxAvgSettle'],
        wRange=NGgraph['wRange'],
        wxAvgRange=NGgraph['wxAvgRange'],
        
        #colormap
        type_color = [colormap[str(x)] for x in NGgraph['year']]
    )
)

In [159]:
p = figure(title="Henry by Week", tools="resize,hover,save",
           x_range=colLabel, y_range=rowLabel)

In [160]:
p.plot_width = 1200
p.toolbar_location = None
p.outline_line_color = None

p.rect("week", "year", 0.9, 0.9, source=source,
       fill_alpha=0.6, color="type_color")

text_props = {
    "source": source,
    "angle": 0,
    "color": "black",
    "text_align": "left",
    "text_baseline": "middle"
}
           
p.text(x="settleX", y="year", text="settle",
       text_font_style="bold", text_font_size="15pt", **text_props)

p.text(x="settleX", y="monthY", text="month",
       text_font_size="9pt", **text_props)

p.text(x="settleX", y="pctY", text="pctChng",
       text_font_size="6pt", **text_props)

p.text(x="settleX", y="avgY", text="wxAvgSettle",
       text_font_size="5pt", **text_props)
           
p.grid.grid_line_color = None
           
p.select_one(HoverTool).tooltips = [
    ("Week's Settlemnt: ", "@settle"),
    ("Prompt Month: ", "@month"),
    ("Percent change in Price: ", "@percChng"),
    ("5 year average price: ", "@wxAvgSettle"),
    ("Range: ", "@wRange"),
    ("5 year average range: ", "@wxAvgRange"),
]

In [161]:
show(p)

In [41]:
rowLabel

['2010', '2011', '2012', '2013', '2014', '2015', '2016']

In [37]:
group_range = [str(x) for x in range(1, 19)]

In [38]:
group_range

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18']

In [47]:
from bokeh.sampledata.periodic_table import elements

In [53]:
elements.head()

,atomic number,symbol,name,atomic mass,CPK,electronic configuration,electronegativity,atomic radius,ion radius,van der Waals radius,...,EA,standard state,bonding type,melting point,boiling point,density,metal,year discovered,group,period
0,1,H,Hydrogen,1.00794,#FFFFFF,1s1,2.200,37.000,NaN,120.000,...,-73.000,gas,diatomic,14.000,20.000,0.000,nonmetal,1766,1,1
1,2,He,Helium,4.002602,#D9FFFF,1s2,nan,32.000,NaN,140.000,...,0.000,gas,atomic,nan,4.000,0.000,noble gas,1868,18,1
2,3,Li,Lithium,6.941,#CC80FF,[He] 2s1,0.980,134.000,76 (+1),182.000,...,-60.000,solid,metallic,454.000,1615.000,0.540,alkali metal,1817,1,2
3,4,Be,Beryllium,9.012182,#C2FF00,[He] 2s2,1.570,90.000,45 (+2),nan,...,0.000,solid,metallic,1560.000,2743.000,1.850,alkaline earth metal,1798,2,2
4,5,B,Boron,10.811,#FFB5B5,[He] 2s2 2p1,2.040,82.000,27 (+3),nan,...,-27.000,solid,covalent network,2348.000,4273.000,2.460,metalloid,1807,13,2


In [14]:
#calc max price last week
HMax = float_formatter(NG[firstn].max())

#return date of start of week
Start = NG.index[0].strftime("%m/%d/%Y")

In [15]:
#return price of first day of week, but not if its nan
NGFirst = NG[np.isfinite(NG[firstn])]
NGFirst = NGFirst[[firstn]]

In [16]:
#calc percent change from first to last, also using shortened NGFirst from above
NGchng = NGFirst[firstn].pct_change(periods=len(NGFirst.index)-1)

In [17]:
#return OI of last entry in COT data (most recent date)
OIlast = OI.ix[-1]

In [18]:
#calc percent change in OI from this week to last week
OIchng = OI['Open Interest'].pct_change(periods=1)

In [19]:
#reformat everything and define dict for display
Week1 = dict(
    Start = [Start],
    Contract = [firstn],
    Open = [float_formatter(NGFirst.ix[0])],
    High = [float_formatter(NGFirst.max())],
    Low = [float_formatter(NGFirst.min())],
    Close = [float_formatter(NGFirst.ix[-1])],
    changePrice = ["{:.2%}".format(NGchng.ix[len(NGchng.index)-1])],
    changeOI  = ["{:.2%}".format(OIchng.ix[len(OIchng.index)-1])],
    OI  = ["{:,}".format(OIlast[0].astype(int))],
    )
    

In [20]:
#bokeh to output nice looking table
source = ColumnDataSource(Week1)

columns = [
        TableColumn(field="Start", title="Week Start"),
        TableColumn(field="Contract", title="Contract"),
        TableColumn(field="Open", title="Open"),
        TableColumn(field="High", title="High"),
        TableColumn(field="Low", title="Low"),
        TableColumn(field="Close", title="Close"),
        TableColumn(field="changePrice", title="%change Price"),
        TableColumn(field="OI", title="OI"),
        TableColumn(field="changeOI", title="%change OI"),
    ]

data_table = DataTable(source=source, columns=columns, width=850, height=75)

show(vform(data_table))

### Storage Report

In [21]:
#api url for series name Weekly Lower 48 States Natural Gas Working Underground Storage, Weekly
storageURL = 'http://api.eia.gov/series/?api_key=B617CC071819064DEFFF9ED570FDB75A&series_id=NG.NW2_EPG0_SWO_R48_BCF.W&out=JSON'

In [22]:
#get requests data from specificied url
storage = requests.get(storageURL)

In [23]:
#the request was for json formatting so uses requests' JSON decoder
storage_json = storage.json()

#storage_json is a nested dict, so take a look at it and find my way into the sub-dict with the data
date_series = storage_json['series'][0]['data']

In [24]:
#loop through that date_series dict and make and array out of the date entries
endi = len(date_series)
date = []

for i in range(endi):
    date.append(date_series[i][0])

In [25]:
#convert that bad boy to a data frame
df = pd.DataFrame(data=date)
df.columns = ['Date']

In [26]:
#loop through the rest of the dict with and put the data in
lenj = len(storage_json)-1

for j in range (lenj):
    data_series = storage_json['series'][0]['data']
    data = []
    endk = len(date_series) 
    for k in range (endk):
        data.append(data_series[k][1])
    df[j] = data

In [27]:
#rename columns
df.columns = ['date','storageUS']

#substring date to use for making a python datevariable, the infer datetime thing didnt work 
df['year'] = df['date'].apply(lambda t: t[0:4])
df['year'] = df['year'].astype(int)

df['month'] = df['date'].apply(lambda t: t[4:6])
df['month'] = df['month'].astype(int)

df['day'] = df['date'].apply(lambda t: t[-2:])
df['day'] = df['day'].astype(int)

In [28]:
#date formatted as datetime
df['date_f'] = pd.to_datetime(df.year*10000 + df.month*100 + df.day, format='%Y%m%d')

In [29]:
#week of year to make y-y comparisons
df['week'] = df['date_f'].dt.week

In [30]:
#move the index to date time
df.set_index(['date_f'], inplace = True)
#sort by datetime acending, bc the data comes in most rescent first
df = df.sort_index()
#calc weekly storage change
df['lastWeek'] = df['storageUS'].shift()
df['change'] = df['storageUS'] - df['lastWeek']

In [31]:
#make it so the index label doesnt appear, just looks prettier
df.index.name = None

In [32]:
#subset the data so for most recent six years
#six because my code to calc 5 year average takes in current week, will use shift and rolling avg to get 5year
df = df[pd.to_datetime(df.year*10000 + df.month*100 + df.day, format='%Y%m%d')>= sixyears]

In [33]:
#rolling average storage volume grouped by week
df['avgChange'] = df.groupby('week')['change'].apply(pd.rolling_mean,len(df),min_periods=1)
df['avgStorageUS'] = df.groupby('week')['storageUS'].apply(pd.rolling_mean,len(df),min_periods=1)


In [34]:
# calc non inclusive five year, sort by week then by date then create var based on shifted avgstorage us
#sort by index back to normal
df = df.sort_values(['week','date'],ascending=[1,1])
df['storage_5yr'] = df['avgStorageUS'].shift()
df = df.sort_index()

In [35]:
#cal percent diff from 5 year average
df['percentDiff'] = (df['storageUS'] - df['storage_5yr'])/df['storage_5yr']

In [36]:
recent = df.tail()
recent

,date,storageUS,year,month,day,week,lastWeek,change,avgChange,avgStorageUS,storage_5yr,percentDiff
2016-02-05,20160205,2864,2016,2,5,5,2934.000,-70.000,-142.000,2488.667,2413.600,0.187
2016-02-12,20160212,2701,2016,2,12,6,2864.000,-163.000,-178.833,2309.833,2231.600,0.210
2016-02-19,20160219,2584,2016,2,19,7,2701.000,-117.000,-141.167,2168.667,2085.600,0.239
2016-02-26,20160226,2536,2016,2,26,8,2584.000,-48.000,-117.167,2051.500,1954.600,0.297
2016-03-04,20160304,2479,2016,3,4,9,2536.000,-57.000,-123.000,1928.500,1818.400,0.363


In [37]:
x = df[df['week']==5]
x

,date,storageUS,year,month,day,week,lastWeek,change,avgChange,avgStorageUS,storage_5yr,percentDiff
2011-02-04,20110204,2144,2011,2,4,5,2355.000,-211.000,-211.000,2144.000,2630.667,-0.185
2012-02-03,20120203,2888,2012,2,3,5,2966.000,-78.000,-144.500,2516.000,2144.000,0.347
2013-02-01,20130201,2684,2013,2,1,5,2802.000,-118.000,-135.667,2572.000,2516.000,0.067
2014-01-31,20140131,1924,2014,1,31,5,2184.000,-260.000,-166.750,2410.000,2572.000,-0.252
2015-01-30,20150130,2428,2015,1,30,5,2543.000,-115.000,-156.400,2413.600,2410.000,0.007
2016-02-05,20160205,2864,2016,2,5,5,2934.000,-70.000,-142.000,2488.667,2413.600,0.187


## Twitter

__some lists for starters__

_nat gas_
* [Peter Gardett](https://twitter.com/petergardett/lists)
* [EIA](https://twitter.com/EIAgov/lists)
* [InterfaxEnergy](https://twitter.com/InterfaxEnergy)
* [Cohan](https://twitter.com/cohan_ds/lists)

_power_
* https://twitter.com/innotap
* https://twitter.com/Brad_E_Foster
* https://twitter.com/rtoinsider
* https://twitter.com/MohitShrestha
* https://twitter.com/nrgytariffxpert


In [38]:
import tweepy

pd.set_option("display.max_rows",1000)
pd.set_option("display.max_columns",50)
pd.set_option("display.max_colwidth",150)

In [39]:
consumer_key = 'UvB8nZGq2m4GcNk64Pv3U9kln'
consumer_secret = 'FhX4W6SV1dGT0GZSBvTzHzt2eoYYS27SrqqKm4CGR3eiqwQURc'

access_token ='18363853-642KR2i2Mq9ZB5aWwZbpziMUX39IfTTdsrlawUXKG'
access_token_secret = 'dwGaU0xEMRJta5Qoos2k47tTEPYR3HwGp0D295nYV6ZxS'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [40]:
public_tweets = api.home_timeline()
for (idx, tweet) in enumerate(public_tweets[0:3]): #First 3 tweets in my public feed
    print ('TWEET %s:\n\n\t%s\n\n' % (idx, tweet.text))

TWEET 0:

	Value Iteration Networks https://t.co/naDDd5wLRs


TWEET 1:

	RT @VICE: The VICE Morning Bulletin: Everything you need to know about the world right now https://t.co/CQ43sfZJFD https://t.co/depnTvDNjT


TWEET 2:

	I will be finishing my 17th marathon, live on the 'periscope' app in 5 mins. Download the app &amp; follow me to watch https://t.co/iWpzL1DEGf




In [41]:
pprint(tweet.__dict__.keys())

dict_keys(['text', '_json', 'retweeted', 'possibly_sensitive', 'in_reply_to_screen_name', 'truncated', 'favorite_count', 'entities', 'user', 'id_str', 'possibly_sensitive_appealable', '_api', 'in_reply_to_user_id_str', 'favorited', 'is_quote_status', 'lang', 'author', 'source_url', 'place', 'created_at', 'in_reply_to_status_id', 'source', 'in_reply_to_user_id', 'extended_entities', 'id', 'retweet_count', 'contributors', 'coordinates', 'geo', 'in_reply_to_status_id_str'])


In [43]:
top10 = api.trends_place(id=2379574)
top10[0]['trends']

[{'name': '#SSAC16',
  'promoted_content': None,
  'query': '%23SSAC16',
  'tweet_volume': None,
  'url': 'http://twitter.com/search?q=%23SSAC16'},
 {'name': '#BenCarson',
  'promoted_content': None,
  'query': '%23BenCarson',
  'tweet_volume': None,
  'url': 'http://twitter.com/search?q=%23BenCarson'},
 {'name': 'Dangerous Woman',
  'promoted_content': None,
  'query': '%22Dangerous+Woman%22',
  'tweet_volume': 262360,
  'url': 'http://twitter.com/search?q=%22Dangerous+Woman%22'},
 {'name': '#HTGAWM',
  'promoted_content': None,
  'query': '%23HTGAWM',
  'tweet_volume': 81096,
  'url': 'http://twitter.com/search?q=%23HTGAWM'},
 {'name': '#FridayFeeling',
  'promoted_content': None,
  'query': '%23FridayFeeling',
  'tweet_volume': 39853,
  'url': 'http://twitter.com/search?q=%23FridayFeeling'},
 {'name': '#IWillSmileTodayBecause',
  'promoted_content': None,
  'query': '%23IWillSmileTodayBecause',
  'tweet_volume': None,
  'url': 'http://twitter.com/search?q=%23IWillSmileTodayBecause'}

In [44]:
for trend in top10[0]['trends']:
    print (trend['name'], trend['url'])

#SSAC16 http://twitter.com/search?q=%23SSAC16
#BenCarson http://twitter.com/search?q=%23BenCarson
Dangerous Woman http://twitter.com/search?q=%22Dangerous+Woman%22
#HTGAWM http://twitter.com/search?q=%23HTGAWM
#FridayFeeling http://twitter.com/search?q=%23FridayFeeling
#IWillSmileTodayBecause http://twitter.com/search?q=%23IWillSmileTodayBecause
Joe's Crab Shack http://twitter.com/search?q=%22Joe%27s+Crab+Shack%22
Chris Hogan http://twitter.com/search?q=%22Chris+Hogan%22
At Trump University http://twitter.com/search?q=%22At+Trump+University%22
Saul Alinsky http://twitter.com/search?q=%22Saul+Alinsky%22
Douglas Adams http://twitter.com/search?q=%22Douglas+Adams%22
Adolphus Washington http://twitter.com/search?q=%22Adolphus+Washington%22
Newcastle http://twitter.com/search?q=Newcastle
Dog Whisperer http://twitter.com/search?q=%22Dog+Whisperer%22
Arnold Donald http://twitter.com/search?q=%22Arnold+Donald%22
Cruise Automation http://twitter.com/search?q=%22Cruise+Automation%22
John Boyega 

In [53]:
results = api.search(q='Henry Hub', count=5, lang='en')
print (type(results))

<class 'tweepy.models.SearchResults'>


In [54]:
for tweet in results:
    print (tweet.created_at, '\n',  tweet.text, '\n\n')

2016-03-11 16:09:29 
 RT @InequalityCU: 23 year old CEO John Henry of @cofoundharlem strives to create diverse businesses to help the Harlem community https://t.… 


2016-03-11 15:23:19 
 @EIAgov reports that Henry Hub natural gas spot prices fell to lowest level in twenty years on Wednesday, to $1.57/MMBtu. 


2016-03-11 15:14:01 
 23 year old CEO John Henry of @cofoundharlem strives to create diverse businesses to help the Harlem community https://t.co/uYy0jpg5oM 


2016-03-11 13:51:46 
 Update Henry Hub Natural Gas Futures  $ 1,84 


2016-03-11 13:08:30 
 NYC - World Trade Center Transportation Hub by mgiachetti https://t.co/5zcZW1IZgz #Photography #fotografia https://t.co/p2QTIZYwwi 




In [64]:
tweets = pd.DataFrame()

tweets["text"] = [tweet.text for tweet in results]
tweets["screen_name"] = [tweet.author.screen_name for tweet in results]
tweets["url"] = [tweet.entities for tweet in results]

tweets

,text,screen_name,url
0,RT @InequalityCU: 23 year old CEO John Henry of @cofoundharlem strives to create diverse businesses to help the Harlem community https://t.…,beedoun,"{'user_mentions': [{'name': 'Cornell Inequality', 'id': 2153427685, 'screen_name': 'InequalityCU', 'id_str': '2153427685', 'indices': [3, 16]}, {'..."
1,"@EIAgov reports that Henry Hub natural gas spot prices fell to lowest level in twenty years on Wednesday, to $1.57/MMBtu.",Foster_Report1,"{'user_mentions': [{'name': 'EIA', 'id': 57111966, 'screen_name': 'EIAgov', 'id_str': '57111966', 'indices': [0, 7]}], 'symbols': [], 'urls': [], ..."
2,23 year old CEO John Henry of @cofoundharlem strives to create diverse businesses to help the Harlem community https://t.co/uYy0jpg5oM,InequalityCU,"{'user_mentions': [{'name': 'cofound harlem', 'id': 2838358228, 'screen_name': 'cofoundharlem', 'id_str': '2838358228', 'indices': [30, 44]}], 'sy..."
3,"Update Henry Hub Natural Gas Futures $ 1,84",davide_del,"{'user_mentions': [], 'symbols': [], 'urls': [], 'hashtags': []}"
4,NYC - World Trade Center Transportation Hub by mgiachetti https://t.co/5zcZW1IZgz #Photography #fotografia https://t.co/p2QTIZYwwi,Henry_Stradford,"{'user_mentions': [], 'media': [{'expanded_url': 'http://twitter.com/Henry_Stradford/status/708278426956914688/photo/1', 'id_str': '70827842649974..."


In [48]:
for tweet in api.search(q='Henry Hub', count=5, show_user=False, lang='en'):
    print (tweet.created_at, '\n',  tweet.text, '\n\n')

2016-03-11 13:51:46 
 Update Henry Hub Natural Gas Futures  $ 1,84 


2016-03-11 13:08:30 
 NYC - World Trade Center Transportation Hub by mgiachetti https://t.co/5zcZW1IZgz #Photography #fotografia https://t.co/p2QTIZYwwi 


2016-03-11 13:07:58 
 VINTAGE ORIGINAL1920-30s PRINT AD OF THE HUB HENRY C.LYTTON &amp; SONS https://t.co/gn1RzFuSMw https://t.co/UNLpchBFav 


2016-03-11 13:03:14 
 "Quality means doing it right when no one is looking." -Henry Ford

Have an awesome Friday, friends! 


2016-03-11 11:58:50 
 RT @restot50: Henry Herbert La Thange https://t.co/kt9EVVDdjz 




In [49]:
results[0].user

User(name='Davide Del Gaudio', _json={'name': 'Davide Del Gaudio', 'notifications': False, 'screen_name': 'davide_del', 'is_translation_enabled': False, 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'default_profile': True, 'verified': False, 'url': None, 'profile_background_color': 'C0DEED', 'followers_count': 224, 'id_str': '389353555', 'geo_enabled': True, 'friends_count': 93, 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/618209419491471361/cBF2Iv1d_normal.jpg', 'lang': 'it', 'profile_link_color': '0084B4', 'protected': False, 'description': 'INDEPENDENT—TRADER : FUTURES—FOREX —COMMODITIES—STOCK—CFD. Money Never Sleeps', 'profile_text_color': '333333', 'following': False, 'statuses_count': 21365, 'contributors_enabled': False, 'profile_use_background_image': True, 'profile_image_url': 'http://pbs.twimg.com/profile_images/618209419491471361/cBF2Iv1d_normal.jpg', 'location': '', 'profile_banner_url': 'https://pbs.twimg.co

In [50]:
new_tweets = api.user_timeline(screen_name='GenscapeWx', count=5)

In [51]:
for tweet in new_tweets:
    print (tweet.text, ' ', tweet.source_url)

Late season New England #snow? @genscape   http://twitter.com
Had enough #rain yet? WPC 5-day outlook suggests additional heavy rains possible to start the weekend. @genscape https://t.co/PAd7JFCC83   http://twitter.com
RT @NWSSanDiego: State reservoir water levels for 12-5-2015 compared to today. Significant improvement. #cawx https://t.co/wKDNJ48b1I   http://twitter.com
It's not often you see radar-derived precipitation totals looking like this. #flooding @genscape @NWSLakeCharles https://t.co/RsRivHq76L   http://twitter.com
Surging levels at Toledo Bend Reservoir (+3ft in 48hr) contributing factor to W. LA #MISO congestion. @genscape https://t.co/xgFFYr0NuO   http://twitter.com


In [52]:
print(new_tweets[0].__dict__.keys())

dict_keys(['text', '_json', 'retweeted', 'in_reply_to_screen_name', 'truncated', 'favorite_count', 'entities', 'user', 'id_str', '_api', 'in_reply_to_user_id_str', 'favorited', 'is_quote_status', 'lang', 'author', 'source_url', 'place', 'created_at', 'in_reply_to_status_id', 'source', 'in_reply_to_user_id', 'id', 'retweet_count', 'contributors', 'coordinates', 'geo', 'in_reply_to_status_id_str'])


### Boneyard

In [ ]:
# C = df.groupby(['week'])['change'].mean()
# T = df.groupby(['week'])['storageUS'].mean()

# #stats = pd.DataFrame(C,T)
# #stats.index.name = 'week'

#calculations done outside the dict
# HLast = float_formatter(NGFirst.ix[-1])
# HFirst = float_formatter(NGFirst.ix[0])
# float_formatter(NGFirst.max())
# float_formatter(NGFirst.min())
# NGchng.ix[len(NGchng.index)-1]
# OIchng.ix[len(OIchng.index)-1]


#"{:,}".format(x[0].astype(int))
#NGMar = NGMar.resample('5D',how='ohlc')

# #rename columns
# NG.columns = [firstn,secondn,thirdn]
# #make it so the index label doesnt appear, just looks prettier
# NG.index.name = None
# #column for week of year
# NG['week'] = NG.index.week

# #cull the columns
# OI = COT[['Open Interest']]

In [127]:
# gb = NG1.groupby('week')

# zscore = lambda x: (x.mean())

# gb.transform(zscore)

# #gb['NG April'].agg([np.min, np.max, np.mean])

# #iterating over group
# # for name, group in gb:
# #     print(name)
# #     print(group)


# # weekly min max
# NG1[firstn+' Weekly Avg'] = NG1.groupby('week')[firstn].apply(pd.rolling_mean,len(NG1),min_periods=1)
    
#df['avgChange'] = df.groupby('week')['change'].apply(pd.rolling_mean,len(df),min_periods=1)

,NG April,NGMay,NGJune
2016-01-14,2.216,2.276,2.340
2016-01-15,2.216,2.276,2.340
2016-01-19,2.204,2.272,2.335
2016-01-20,2.204,2.272,2.335
2016-01-21,2.204,2.272,2.335
2016-01-22,2.204,2.272,2.335
2016-01-25,2.257,2.311,2.363
2016-01-26,2.257,2.311,2.363
2016-01-27,2.257,2.311,2.363
2016-01-28,2.257,2.311,2.363
